In [1]:
%matplotlib qt5
from process_files import *
from func_science import *

qt.qpa.xcb: X server does not support XInput 2


qt.qpa.xcb: QXcbConnection: XCB error: 1 (BadRequest), sequence: 169, resource id: 132, major code: 130 (Unknown), minor code: 47


In [3]:
#
line = 'Fe_I_6173'
config = configobj.ConfigObj('config.ini')
dkdir = config['darks']['directory']
scdir = config['science']['directory']
settings = [f for f in os.listdir(scdir) if 'settings' in f]
settings = scdir + os.sep + settings[0]
settings = configobj.ConfigObj(settings)
# Other numbers
linestr = 'Line_' + str(get_line_num(settings, line, 0))
nacc = int(settings[linestr]['Polarimeter\\NAccumulations'])
nwav = int(settings[linestr]['NWavePoints'])
filtstr = settings[linestr]['Filter']
modstr = settings[linestr]['Polarimeter\\Modulation']
nmod = int(settings[filtstr]['Polarimeter\\'+modstr+'\\NModulations'])
nfpc = nmod*nacc*nwav
nfpw = nmod*nacc
ncyc = len(os.listdir(scdir+os.sep+line))//3
wave_range = np.float64(settings[linestr]['WaveScanRange'])
wave_step = wave_range/(np.float64(settings[linestr]['NWavePoints'])-1)

In [5]:
# alignment
rotang1 = np.float64(config['targetplate']['rotang1'])
rotang2 = np.float64(config['targetplate']['rotang2'])
#
shift1_y = np.float64(config['targetplate'][line+'/shift1_y'])
shift1_x = np.float64(config['targetplate'][line+'/shift1_x'])
shift2_y = np.float64(config['targetplate'][line+'/shift2_y'])
shift2_x = np.float64(config['targetplate'][line+'/shift2_x'])
pol1_shifts = [shift1_y, shift1_x, 0, 0]
pol2_shifts = [shift2_y, shift2_x, 0, 0]
# flats
fa1_name = config['flats'][line+'/pol1']
fa2_name = config['flats'][line+'/pol2']
ff_name = config['flats'][line+'/pol']
line_cens = pf.open(ff_name)[1].data
print('Master flat read from ', ff_name)
#
l0dir = config['science'][line+'/l0dir']
pol1_names = [l0dir+os.sep+f for f in sorted(os.listdir(l0dir+os.sep+'pol1')) if 'pol1' in f]
pol2_names = [l0dir+os.sep+f for f in sorted(os.listdir(l0dir+os.sep+'pol2')) if 'pol2' in f]

Master flat read from  /arch/1/VTT/hellride/raw/20230502/L0/HELLRIDE_pol_20230502_Flats_1309_Fe_I_6173_fm.FITS


In [78]:
# Get science data
for i in range(1):
    sc1 = pf.open(pol1_names[i])[0].data
    sc2 = pf.open(pol2_names[i])[0].data
    

In [ ]:
def compute_line_minima(lines):
    Y, X, Z = lines.shape
    minima = np.zeros([Y, X])
    for y in tqdm.tqdm(range(Y)):
        for x in range(X):
            minima[y,x] = find_line_valley(lines[y,x])
    return minima

In [ ]:
sc1_ = compute_line_minima(sc1[:,:,0])

In [67]:


from lmfit.models import GaussianModel, ConstantModel, QuadraticModel

def absorption_line(xdata, cont, cen, sig, amp):
    return cont*(1 - amp*np.exp(-0.5*(xdata-cen)**2/sig**2))

def fit_absorption_line(spectral_line, line_width=3.0, gain_factor=1.0, plot=False):
    """
    Compute best fit for the line+continuum in the form of f(x)*[1-g(x)] where
    x is the pixel number starting at 0
    f(x) is quadratic (3 params)
    g(x) is Gaussian (3 params)
    Input:  1d-array spectral line
            (opt.) estimated width of the line
            (opt.) gain factor to estimate weights 
            (opt.) plot the results
    Output: list of inital guess parameters
            list of best fit parameters
    """
    xdata = np.arange(len(spectral_line))
    ydata = np.float64(spectral_line)
    ydata_error = np.sqrt(spectral_line.mean()/gain_factor)
    #
    cont_guess = ydata[0]
    amp_guess = 1-ydata.min()/cont_guess
    cen_guess = xdata[np.argwhere(ydata==ydata.min())].flatten()[0]
    sig_guess = line_width/2.355
    #
    guess_params = Parameters()
    guess_params.add(name='amp', value=amp_guess, max=1)
    guess_params.add(name='cen', value=cen_guess, min=xdata.min(), max=xdata.max())
    guess_params.add(name='sig', value=sig_guess, min=0, max=xdata.max())
    guess_params.add(name='cont', value=cont_guess)
    #
    model = Model(absorption_line)
    result = model.fit(ydata, guess_params, xdata=xdata)
    if(plot):
        fig, ax = plt.subplots(1,1)
        ax.plot(ydata, 'k--')
        ax.plot(result.init_fit, 'k:')
        ax.plot(result.best_fit, 'k-')
        ax.legend(['Data', 'Initial', 'Fitted'])
    return result

def find_line_cens(lines, beg=0, end=11):
    Y, X, Z = lines.shape
    cens = np.zeros([Y, X])
    for y in tqdm.tqdm(range(Y)):
        for x in range(X):
            res = fit_absorption_line(lines[y,x,beg:end], plot=False)
            cens[y, x] = res.best_values['cen']
    return cens

In [ ]:
def fit_absorption_line(ydata, plot=False):
    xdata = np.arange(len(ydata))
    #
    cont_guess = ydata[0]
    amp_guess = cont_guess-ydata.min()
    cen_guess = xdata[np.argwhere(ydata==ydata.min())].flatten()[0]
    sig_guess = 2
    #
    guess_params = Parameters()
    guess_params.add(name='amplitude', value=amp_guess, max=cont_guess)
    guess_params.add(name='center', value=cen_guess, min=xdata.min(), max=xdata.max())
    guess_params.add(name='sigma', value=sig_guess, min=0, max=xdata.max())
    guess_params.add(name='c', value=cont_guess)
    #
    cont = ConstantModel()
    absl = GaussianModel()
    model = cont - absl
    res = model.fit(ydata, guess_params, x=xdata)
    if (plot==True):
        plt.figure()
        plt.plot(res.best_fit)
        plt.plot(ydata)
    cen = res.best_values['center']
    return cen

def find_line_cens(lines, beg=0, end=11):
    Y, X, Z = lines.shape
    cens = np.zeros([Y, X])
    for y in tqdm.tqdm(range(Y)):
        for x in range(X):
            cens[y, x] = fit_absorption_line(lines[y,x,beg:end], plot=False)
    return cens

In [68]:
cens1 = find_line_cens(sc1[:,:,0])

100%|██████████| 1280/1280 [1:50:03<00:00,  5.16s/it]


In [1]:
def compute_doppler_fft(dref, dsci):
    Y, X, Z = dref.shape
    dref_fft, dsci_fft = np.fft.fft(dref, axis=2), np.fft.fft2(dsci, axis=2)
    d_corr = np.fft.ifftshift(np.abs(np.fft.ifft(dref_fft*np.conj(dsci_fft))), axes=2)
    return d_corr
    # sh = np.argwhere(d_corr==d_corr.amax(axis=2))-Z//2

In [71]:
plt.imshow(cens1-line_cens[:,:,0], cmap='RdBu')

In [87]:
show_img_series(sc1[:,:,3]/sc1[:,:,0], fps=1)